In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import os

import itertools

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

In [4]:
cfg.config

{'out_dir': '/Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out',
 'LH_sas_dir': '/Volumes/neuro/labs/grantlab/research/HIE_NRN_data/transferred_from_RTI/Late_Hypothermia/Clinical_Data',
 'OC_sas_dir': '/Volumes/neuro/labs/grantlab/research/HIE_NRN_data/transferred_from_RTI/Optimizing_Cooling/Clinical Data',
 'LH_follow_up_sas_dir': '/Volumes/neuro/labs/grantlab/research/HIE_NRN_data/transferred_from_RTI/Late_Hypothermia_Follow-up/Clinical Data',
 'OC_follow_up_sas_dir': '/Volumes/neuro/labs/grantlab/research/HIE_NRN_data/transferred_from_RTI/Optimizing_Cooling_Follow-up/Clinical Data',
 'LH_analysis_sas_dir': '/Volumes/neuro/labs/grantlab/research/HIE_NRN_data/transferred_from_RTI/Late_Hypothermia/Analysis_Data/Data Files',
 'OC_analysis_sas_dir': '/Volumes/neuro/labs/grantlab/research/HIE_NRN_data/transferred_from_RTI/Optimizing_Cooling/Analysis Data/Data Files',
 'LH_dir': '/Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/csv/LH/ClinicalData',
 'OC_dir': '/Vo

## 02. filename

In [5]:
out_dir = cfg.config['out_dir']
outcome_filename = f'{out_dir}/out-merged-normalized/30-02-outcome.csv'

df = pd.read_csv(outcome_filename)

In [6]:
df['disabilityLevel']

0      severe
1      severe
2      normal
3        mild
4      normal
        ...  
527       NaN
528    normal
529    severe
530    normal
531    normal
Name: disabilityLevel, Length: 532, dtype: object

In [7]:
is_death_before_followup = df['deathBeforeFollowup'].copy().fillna(False)

/var/folders/84/vxdpc5mx2cqdr6zvp8p_tygc0000gn/T/ipykernel_16826/3580702884.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_death_before_followup = df['deathBeforeFollowup'].copy().fillna(False)


In [8]:
is_death_before_followup

0       True
1      False
2      False
3      False
4      False
       ...  
527    False
528    False
529     True
530    False
531    False
Name: deathBeforeFollowup, Length: 532, dtype: bool

In [9]:
df['disabilityLevelDeath'] = df['disabilityLevel'].copy()
df.loc[is_death_before_followup, 'disabilityLevelDeath'] = 'death'

In [10]:
columns = ['disabilityLevel', 'deathBeforeFollowup', 'disabilityLevelDeath']
df[columns]

,disabilityLevel,deathBeforeFollowup,disabilityLevelDeath
0,severe,True,death
1,severe,False,severe
2,normal,False,normal
3,mild,False,mild
4,normal,False,normal
...,...,...,...
527,NaN,False,NaN
528,normal,False,normal
529,severe,True,death
530,normal,False,normal


In [11]:
columns = ['_study', 'center', 'subjectID', 'uniqueID', 'followupCenter', 'disabilityLevelDeath']

df_out = df[columns]
for idx in ['center', 'followupCenter']:
    df_out[idx] = df_out[idx].astype('O')
    df_out.loc[:, idx] = df_out[idx].apply(COMBINE_harmonizer.to_center)

out_filename = f'{out_dir}/out-merged-normalized/31-05-disability-level-death.csv'
df_out.to_csv(out_filename, index=False)